In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils.data_utils import (
    test_missing_bias,
    get_missing_data_summary,
    missing_by_dataset,
    missing_by_outcome,
    handle_impossible_zeros,
    impute_missing_values,
)

from src.utils.data_utils import drop_id
from src.utils.graph_utils import(
    display_relative_histoplot,
    display_histoplot,
    display_multi_histoplot,
    display_multi_boxplot,
    display_categorical_plots,
    display_boxplot,
    display_scatterplot
) 
from src.utils.notebook_setup import setup_notebook_environment

# Quick setup
dbs, logger = await setup_notebook_environment()

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Now ready to work
logger.info("Starting data preprocessing...")

# Checking Bronze Layer
bronze_data = await dbs.get_bronze_data()
bronze_data_df = pd.DataFrame(bronze_data)
bronze_data_df.head(10)

In [ ]:


display_histoplot(df=bronze_data_df, column_name='age', title='Age Distribution Overall')
logger.info(f"Mean: {bronze_data_df['age'].mean():.2f}")
logger.info(f"Median: {bronze_data_df['age'].median():.2f}")
logger.info(f"Mode: {bronze_data_df['age'].mode()[0]:.2f}")

display_relative_histoplot(df=bronze_data_df, x_column_name='age', hue_column_name='sex', title='Age Distribution by Sex')
bronze_data_df['sex'].value_counts()

# see age distribution by dataset
display_relative_histoplot(bronze_data_df, 'age', 'dataset', 'Age Distribution by Dataset')

In [ ]:
# Identify Duplicates
bronze_data_df.duplicated().sum()

if bronze_data_df.duplicated().sum() > 0:
    bronze_data_df = bronze_data_df.drop_duplicates()
    logger.info("Dropped duplicate rows")
else:
    logger.info("No duplicate rows found")

# checking data info
bronze_data_df.info()
logger.info(bronze_data_df.info())

# DROP COLUMNS
bronze_data_df = bronze_data_df.drop(columns=['_id', "id"])

# show dataframe
bronze_data_df.head()

In [ ]:
# Checing Categorical Columns
cat_cols = bronze_data_df.select_dtypes(include=['object']).columns
num_cols = bronze_data_df.select_dtypes(include=['int64', 'float64']).columns

# Check unique values in categorical columns
logger.info("Unique value COUNT in categorical columns")
logger.info(bronze_data_df[cat_cols].nunique())

logger.info("Unique values in categorical columns")
logger.info(f"ca: {bronze_data_df['ca'].unique()[:bronze_data_df['ca'].nunique()]}")
logger.info(f"thal: {bronze_data_df['thal'].unique()[:bronze_data_df['thal'].nunique()]}")
logger.info(f"slope: {bronze_data_df['slope'].unique()[:bronze_data_df['slope'].nunique()]}")

# Check numerical columns
logger.info("Unique values COUNT in numerical columns")
logger.info(bronze_data_df[num_cols].nunique())
# Remove non-plotting columns
cat_cols = [col for col in cat_cols if col not in ['_id', 'dataset']]
num_cols = [col for col in num_cols if col not in ['id']]

logger.info("Unique value COUNT in categorical columns")
logger.info(bronze_data_df[cat_cols].nunique())

# Visualize Numerical Columns (histograms)
if len(num_cols) > 6:
    display_multi_histoplot(bronze_data_df, num_cols[:6])
    if len(num_cols) > 6:
        display_multi_histoplot(bronze_data_df, num_cols[6:])
else:
    display_multi_histoplot(bronze_data_df, num_cols)

# Visualize Numerical Boxplots (fixed)
display_multi_boxplot(bronze_data_df, num_cols, target_col='num')

# Visualize Categorical Columns (use count plots instead)
display_categorical_plots(bronze_data_df, cat_cols, target_col='num')



In [ ]:
missing_df = get_missing_data_summary(bronze_data_df)

missing_df.head(20)

In [ ]:
# Running Tests Missing Data to Determine MNAR or MAR Missingness

# TRUE = Missing pattern correlates with heart disease outcome, most likely missing because of patient condition (sicker patients get more tests)

missing_bias_df = pd.DataFrame(columns=['Column', 'MNAR from Chi-square test'])
for col in bronze_data_df.columns:
    missing_bias_df.loc[len(missing_bias_df)] = [col, test_missing_bias(bronze_data_df, col, 'num')]

logger.info("Missing bias:")
missing_bias_df.head(20)

In [ ]:
# Running Tests Missing Data to Determine MNAR or MAR Missing by Dataset

# TRUE = Different studies have different missing patterns, Different hospitals/studies had different protocols
missing_by_dataset_df = pd.DataFrame(columns=['Column', 'MNAR FROM DATASET ORIGIN'])
for col in bronze_data_df.columns:
    missing_by_dataset_df.loc[len(missing_by_dataset_df)] = [col, missing_by_dataset(bronze_data_df, col)]

logger.info("Missing by dataset:")
missing_by_dataset_df.head(20)

In [ ]:
# Check missingness by outcome

missing_by_outcome_df = pd.DataFrame(columns=['Column', 'MNAR FROM OUTCOME'])
for col in bronze_data_df.columns:
    missing_by_outcome_df.loc[len(missing_by_outcome_df)] = [col, missing_by_outcome(bronze_data_df, col)]

logger.info("Missing by outcome:")
missing_by_outcome_df.head(20)

In [ ]:
# Create binary target as required
bronze_data_df['target'] = (bronze_data_df['num'] > 0).astype(int)
# 0 = no heart disease, 1 = any heart disease

# Drop original multi-class target  
bronze_data_df = bronze_data_df.drop(columns=['num'])
bronze_data_df.head(20)


In [ ]:
"""
Judging from the MNAR tests and the missing patterns, we can see that ca and thal values should be dropped
 - Medical knowledge confirms that 'ca' being fluoroscopy is an expensive and invasive test only order for most likely high risk paitents
 - Medical knowledge confirms that 'thal' being a stress test is only conducted when doctors suspect heart problems
 - TRUE across all tests indicated that Systematic, non-random missingness
 - 66.4% and 52.8% missing = Too much missing data
 - Missingness predicts outcome, so we should drop these columns

 Imputing with mean or mode would introdue bias!
"""

# Drop ca and thal columns
logger.info("Dropping ca and thal columns!")
if 'ca' in bronze_data_df.columns and 'thal' in bronze_data_df.columns:
    bronze_data_df = bronze_data_df.drop(columns=['ca', 'thal'])

logger.info(bronze_data_df.shape)
bronze_data_df.head()

In [ ]:
"""
Slope is the slope of the peak exercise ST segment. It refers to the excercise stress test and when its administered it can be a valuable test
Now for the slope column ... there was about 33% missing data meaning that 66% of the data was complete. 

Stress test can be a valuable test, but it is not always administered. But even when its not administered, it can be an indicator that it was Not needed

So Since 
"""

# Create a new column for missing slope values
bronze_data_df['slope'] = bronze_data_df['slope'].fillna('not_tested')
logger.info(bronze_data_df.shape)
bronze_data_df.head()


# Find missing values again
missing_values = get_missing_data_summary(bronze_data_df)
logger.info("Missing values after handling slope missing values")
missing_values.head(20)

In [ ]:


# 1. Handle impossible zeros
cleaned_df = handle_impossible_zeros(bronze_data_df)

# 2. Impute all missing values (including new NaNs from zeros)
imputed_df = impute_missing_values(cleaned_df)

# 3. Verify no missing values remain : summing the null counts across all columns
assert imputed_df.isnull().sum().sum() == 0, "Still have missing values!"


In [ ]:

# Final Step: Save cleaned data to Silver layer

# Verify final data quality
logger.info("=" * 60)
logger.info("FINAL SILVER LAYER DATA SUMMARY")
logger.info("=" * 60)

logger.info(f"Final dataset shape: {imputed_df.shape}")
logger.info(f"Missing values remaining: {imputed_df.isnull().sum().sum()}")
logger.info(f"Target variable distribution:\n{imputed_df['target'].value_counts()}")

# Show final columns
logger.info(f"Final columns: {list(imputed_df.columns)}")

# Save to Silver collection in MongoDB
logger.info("Saving cleaned data to Silver layer...")

# Convert DataFrame to records for MongoDB
silver_data = imputed_df.to_dict('records')

# Insert into Silver collection
success = await dbs.insert_silver_data(silver_data)

if success:
    logger.info(f"Successfully saved {len(silver_data)} records to Silver layer")
    
    # Verify the save
    silver_count = await dbs.get_collection_count('silver')
    logger.info(f"Silver collection now contains: {silver_count} documents")
    
    # Show sample of Silver data
    sample_silver = await dbs.get_sample_records('silver', count=3)
    logger.info("Sample Silver layer records:")
    for i, record in enumerate(sample_silver):
        logger.info(f"Record {i+1}: {record}")
        
else:
    logger.error("Failed to save data to Silver layer")

logger.info("Data preprocessing complete! Ready for feature engineering.")